In [125]:
import pandas as pd
import rpy2
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from rpy2.robjects import r, default_converter
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import pandas2ri

In [126]:
# LOAD BOMBUS VISIT TIMES, EXTACT SPECIES ABUNDANCE DISTRIBUTION
visit_durations_clover = pd.read_csv('csvs/visit_durations_clover.csv', parse_dates=['visit_start', 'visit_end'])
species_count = visit_durations_clover.value_counts('species').reset_index()
species_count = species_count[species_count['species'] != 'unk']
print(species_count)

        species  count
0  vosnesenskii    173
1      fervidus     70
3  griseocollis     13
4    nevadensis      2
5     appositus      1
6        mixtus      1


In [127]:
cnt_visits_per_hr = pd.read_csv('csvs/cnt_visits_per_hour.csv', parse_dates=['date'])

In [128]:
# FIT BASELINE POISSON MODEL

cnt_visits_per_hr['log_hours'] = np.log(cnt_visits_per_hr['hours']) 

model_pois = smf.glm(
    formula="visit_count ~ 1", 
    data=cnt_visits_per_hr,
    family=sm.families.Poisson(),
    offset=cnt_visits_per_hr['log_hours']
).fit()

pearson_chi2 = sum(model_pois.resid_pearson**2) ## CHECK FOR OVERDISPERSION (>1.5?)
dispersion = pearson_chi2 / model_pois.df_resid
print("Dispersion =", dispersion)
if dispersion > 1.5:
    print("OVERDISPERSED USE NEG BIONOMIAL")

Dispersion = 3.316910978226268
OVERDISPERSED USE NEG BIONOMIAL


In [129]:
# FIT NEGATIVE BINOMIAL MODEL DUE TO OVERDISPERSION
model_nb = smf.glm(
    formula="visit_count ~ 1",
    data=cnt_visits_per_hr,
    family=sm.families.NegativeBinomial(),
    offset=cnt_visits_per_hr['log_hours']
).fit()

print(model_nb.summary())

rate_per_hour = np.exp(model_nb.params['Intercept'])

print("\nEstimated visit rate per hour:", rate_per_hour)
print()

# CHECK FOR ZERO-INFLATION
observed_zero_prop = (cnt_visits_per_hr['visit_count'] == 0).mean()
mu = model_nb.predict()
alpha = model_nb.scale 
predicted_zero_prob = np.mean((1 + alpha * mu) ** (-1/alpha))

print("Observed zero proportion:", observed_zero_prop)
print("Predicted zero (NB) proportion:", predicted_zero_prob)
print()
if predicted_zero_prob > observed_zero_prop:
    print('NO ZERO INFLATION NECESSARY')

                 Generalized Linear Model Regression Results                  
Dep. Variable:            visit_count   No. Observations:                   35
Model:                            GLM   Df Residuals:                       34
Model Family:        NegativeBinomial   Df Model:                            0
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -74.101
Date:                Wed, 15 Oct 2025   Deviance:                       31.951
Time:                        16:43:05   Pearson chi2:                     30.1
No. Iterations:                     5   Pseudo R-squ. (CS):              0.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.4204      0.201     -7.070      0.0

/home/misha/miniconda3/envs/bee_mapping/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [130]:

mu_14h = 0.24161904774478468 * 14 # RATE * 14 HOURS
species_list = species_count['species'].values
p_species = species_count['count'] / species_count['count'].sum()

def rneg_binomial(mu): # NEG BINOMIAL DRAW FUNCTION
    # Var = mu + alpha*mu^2 (alpha=1)
    p = 1 / (1 + mu) # success probability
    return np.random.negative_binomial(1, p)

In [ ]:

n_iter = 1000

bootstrap_results = []

for b in range(n_iter):
    total_visits = max(rneg_binomial(mu_14h), 0)

    if total_visits > 0:
        sim_counts = np.random.multinomial(total_visits, p_species)
    else:
        sim_counts = np.zeros_like(p_species, dtype=int)

    bootstrap_results.append(dict(zip(species_list, sim_counts)))

sim_df = pd.DataFrame(bootstrap_results)
sim_df

,vosnesenskii,fervidus,griseocollis,nevadensis,appositus,mixtus
0,5,2,0,1,0,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,5,3,0,0,1,1
4,0,0,0,0,0,0
...,...,...,...,...,...,...
995,0,0,0,0,0,0
996,1,0,0,0,0,0
997,1,1,0,0,0,0
998,2,0,0,0,0,0


In [132]:
iNEXT = importr("iNEXT")
from rpy2.robjects import FloatVector

def inext_coverage_estimator(row):
    counts = row[row > 0]
    if len(counts) == 0:
        return 0.0

    with localconverter(default_converter + pandas2ri.converter):
        r_counts = pandas2ri.py2rpy(counts)

    out = iNEXT.iNEXT(r_counts, q=FloatVector([0]), datatype="abundance")
    data_info = out.rx2("DataInfo")
    coverage = data_info.rx2("SC")[0]
    return float(coverage)

In [133]:
sim_df['coverage'] = sim_df.apply(inext_coverage_estimator, axis=1)


In [134]:
sim_df['coverage']

0      0.9028
1      1.0000
2      1.0000
3      0.8364
4      0.0000
        ...  
995    0.0000
996    1.0000
997    0.6667
998    1.0000
999    0.0000
Name: coverage, Length: 1000, dtype: float64

In [135]:
sim_df['coverage'].describe()

count    1000.000000
mean        0.723510
std         0.404425
min         0.000000
25%         0.666700
50%         1.000000
75%         1.000000
max         1.000000
Name: coverage, dtype: float64

In [136]:
from rpy2.robjects import IntVector, FloatVector
from rpy2.robjects.packages import importr
from rpy2.rinterface_lib.embedded import RRuntimeError

iNEXT = importr("iNEXT")

def rneg_binomial(mu):
    """Negative binomial draw with Var = mu + mu^2 (alpha = 1)."""
    p = 1.0 / (1.0 + mu)
    return np.random.negative_binomial(1, p)

def simulate_one_day(mu_14h, p_species):
    """Simulate one 14-hour day of visits and allocate to species."""
    total_visits = max(rneg_binomial(mu_14h), 0)
    if total_visits > 0:
        return np.random.multinomial(total_visits, p_species).astype(int)
    else:
        return np.zeros_like(p_species, dtype=int)

def inext_coverage_from_counts(counts_np):
    """
    Compute sample coverage with iNEXT (q=0) from a pooled abundance vector.
    Robust: handles empty / degenerate inputs; falls back to Good's estimator.
    """
    N = int(counts_np.sum())
    if N == 0:
        return 0.0  # no observations ⇒ zero coverage

    # Keep only observed species (iNEXT expects non-zero abundances)
    nz = counts_np[counts_np > 0].astype(int)
    if nz.size == 0:
        return 0.0

    # Try iNEXT; if it fails (edge cases), fall back to Good's coverage.
    try:
        r_counts = IntVector(list(nz))
        out = iNEXT.iNEXT(r_counts, q=FloatVector([0.0]), datatype="abundance")
        data_info = out.rx2("DataInfo")
        sc = float(list(data_info.rx2("SC"))[0])  # sample coverage
        # Guard against NaN/None from R edge cases
        if np.isnan(sc):
            raise ValueError("iNEXT returned NaN coverage")
        return sc
    except Exception:
        # Fallback = Good’s coverage: C-hat = 1 - F1 / N
        F1 = int(np.sum(nz == 1))
        return 1.0 - (F1 / float(N))

def bootstrap_coverage_by_days(
    days_to_test, n_iter, mu_14h, p_species, random_seed=None
):
    """
    For each D in days_to_test:
      - simulate D independent 14-hour days per bootstrap iteration,
      - pool counts across days,
      - compute coverage (iNEXT with robust fallback),
      - summarize mean + 95% CI across iterations.
    Returns a tidy DataFrame.
    """
    if random_seed is not None:
        np.random.seed(random_seed)

    results = []
    for D in days_to_test:
        covs = np.empty(n_iter, dtype=float)
        for b in range(n_iter):
            pooled = np.zeros_like(p_species, dtype=int)
            for _ in range(D):
                pooled += simulate_one_day(mu_14h, p_species)
            covs[b] = inext_coverage_from_counts(pooled)

        mean_cov = float(np.mean(covs))
        ci_low, ci_high = np.quantile(covs, [0.025, 0.975])
        results.append({
            "days": D,
            "mean_coverage": mean_cov,
            "ci_low_95": float(ci_low),
            "ci_high_95": float(ci_high)
        })

    return pd.DataFrame(results).sort_values("days").reset_index(drop=True)


days_to_test = [1, 2, 3, 5, 7, 10]
n_iter = 1000 

coverage_summary = bootstrap_coverage_by_days(
    days_to_test=days_to_test,
    n_iter=n_iter,
    mu_14h=mu_14h,
    p_species=p_species.values if hasattr(p_species, "values") else p_species,
    random_seed=42
)

coverage_summary

,days,mean_coverage,ci_low_95,ci_high_95
0,1,0.710911,0.000000,1.0
1,2,0.900000,0.000000,1.0
2,3,0.948858,0.666700,1.0
3,5,0.974580,0.805103,1.0
4,7,0.984474,0.889700,1.0
5,10,0.986400,0.913360,1.0


In [137]:
coverage_summary.to_csv('csvs/boot_out_1k.csv')